In [2]:
import requests
import csv

def get_country_info(accession_numbers, output_csv):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    db = "nuccore"
    rettype = "gbwithparts"  # Modified to retrieve GenBank records with features
    retmode = "text"

    with open(output_csv, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["Accession Number", "Country"])

        for accession_number in accession_numbers:
            url = f"{base_url}?db={db}&id={accession_number}&rettype={rettype}&retmode={retmode}"
            response = requests.get(url)

            if response.status_code == 200:
                record_lines = response.text.splitlines()
                organism_line = next((line for line in record_lines if "ORGANISM" in line), None)

                if organism_line:
                    country = ''
                    feature_start = False
                    for line in record_lines:
                        if line.startswith("FEATURES"):
                            feature_start = True
                            continue
                        elif line.startswith("ORIGIN"):
                            break

                        if feature_start:
                            if "/country" in line.lower():
                                country += line.strip()[12:]  # Extract country information

                    csv_writer.writerow([accession_number, country.strip()])
                else:
                    csv_writer.writerow([accession_number, "Country not found"])

            else:
                csv_writer.writerow([accession_number, f"Error fetching data. Status code: {response.status_code}"])

if __name__ == "__main__":
    # Replace "/path/to/accession_numbers_file.txt" and "/path/to/output_genbank_info.csv" with the actual file paths
    with open("/path/to/SequenceList.txt.txt", "r") as file:
        accession_numbers = [line.strip() for line in file]

    get_genbank_info(accession_numbers, "/path/to/output.csv")
